```
Test set: Average loss: 0.1306, Accuracy: 9596/10000 (96%)

Train Epoch: 27 [0/60000 (0%)]	Loss: 0.310591
Train Epoch: 27 [10000/60000 (17%)]	Loss: 0.297366
Train Epoch: 27 [20000/60000 (33%)]	Loss: 0.270111
Train Epoch: 27 [30000/60000 (50%)]	Loss: 0.416887
Train Epoch: 27 [40000/60000 (67%)]	Loss: 0.339068
```

96% 的准确率, 暂时无法提高了


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    100, shuffle=True)


log_interval  = 100

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        print x.size()
        x = F.relu(self.fc1(x))
        print x.size()
        x = F.dropout(x, training=self.training)
        print x.size()
        x = self.fc2(x)
        print x.size()
        return F.log_softmax(x)
    
model = Net()
model = torch.load('./mnist.model')    

optimizer = optim.SGD(model.parameters(), lr=0.0001)



def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        print data.size()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

        return
  
train(1)
    
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    
# for epoch in range(1, 100):
#     train(epoch)
#     test(epoch)
#     torch.save(model,'./mnist.model')


torch.Size([100, 1, 28, 28])
torch.Size([100, 10, 24, 24])
torch.Size([100, 10, 12, 12])
torch.Size([100, 20, 4, 4])
torch.Size([100, 320])
torch.Size([100, 50])
torch.Size([100, 50])
torch.Size([100, 10])
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.495733
